In [1]:
import os, sys
import importlib
import numpy as np
import torch
import yaml

USER = os.environ.get('USER')
HOME = os.environ.get('HOME')
print('Hello, {}!'.format(USER))

WORKSHOP_DIR = '{}/icarus_ml_workshop_2023/'.format(HOME)
if not os.path.exists(WORKSHOP_DIR):
    print('Could not find WORKSHOP_DIR {}. Trying recursive find relative to {}'.format(WORKSHOP_DIR, HOME))
    import subprocess

    # Run the find command
    command = 'find $HOME -type d -name "icarus_ml_workshop_2023"'
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Store the output in a variable
    output = result.stdout.strip()
    
    if output == '':
        raise ValueError('Workshop directory not found. Please configure the WORKSHOP_DIR variable in this cell')

    # Print the output
    print('Found workshop dir at {}'.format(output))
    
    WORKSHOP_DIR = output + '/'

else:
    print('Using WORKSHOP_DIR {}'.format(WORKSHOP_DIR))

MATCHA_DIR = '/sdf/group/neutrino/{}/matcha'.format(USER)
IS_S3DF = False
if os.path.exists('/sdf/data/neutrino/{}'.format(USER)):
    print('You are on S3DF.')
    IS_S3DF = True
else:
    print('You are on SDF.')

Hello, amogan!
Could not find WORKSHOP_DIR /sdf/home/a/amogan/icarus_ml_workshop_023/. Trying recursive find relative to /sdf/home/a/amogan
Found workshop dir at /sdf/home/a/amogan/icarus_ml_workshop_2023
You are on S3DF.


In [2]:
# Install Andrew's fork of lartpc_mlreco3d if on SDF
SDF_SOFTWARE_DIR = ''
if not IS_S3DF:
    os.chdir('/sdf/group/neutrino/{}/'.format(USER))
    os.system('git clone https://github.com/andrewmogan/lartpc_mlreco3d.git lartpc_mlreco3d_andrew_fork')
    SDF_SOFTWARE_DIR = '/sdf/group/neutrino/{}/lartpc_mlreco3d_andrew_fork/'

In [3]:
# Check if matcha is installed. If not, install it in /sdf/group/neutrino/$USER/.
package_name = 'matcha'

try:
    importlib.import_module(package_name)
    print('matcha import successful.')
except ImportError:
    print('matcha import failed. Attempting to install from Github...')
    os.chdir('/sdf/group/neutrino/{}/'.format(USER))
    os.system('git clone https://github.com/andrewmogan/matcha.git')
    os.chdir('matcha')
    os.system('python3 -m pip install .')
    try:
        # Try again
        importlib.import_module(package_name)
        print('matcha import successful.')
    except:
        raise ImportError('matcha still not found. Time to panic.')

matcha import successful.


In [4]:
# Check and set necessary paths
SOFTWARE_DIR = '/sdf/group/neutrino/amogan/lartpc_mlreco3d_myfork/'
if not IS_S3DF:
    SOFTWARE_DIR = SDF_SOFTWARE_DIR
CONFIG_PATH = WORKSHOP_DIR+'post_processors/crt_tpc_matching/'
ANA_CONFIG = CONFIG_PATH + 'crt_tpc_ana_config.cfg'
MODEL_CONFIG = CONFIG_PATH + 'dhk_config.cfg'

# Check directories 
if not os.path.exists(SOFTWARE_DIR):
    raise ValueError('Software directory {:s} does not exist'.format(SOFTWARE_DIR))
if not os.path.exists(CONFIG_PATH):
    raise ValueError('Config directory {:s} does not exist'.format(CONFIG_PATH))
if not os.path.exists(ANA_CONFIG):
    raise FileNotFoundError('Ana config file {:s} does not exist'.format(ANA_CONFIG))
if not os.path.exists(MODEL_CONFIG):
    raise FileNotFoundError('Model config file {:s} does not exist'.format(MODEL_CONFIG))
    
sys.path.append(SOFTWARE_DIR)

In [5]:
from mlreco.main_funcs import process_config

# Load anatools config file
DATA_PATH = '/sdf/data/neutrino/icarus/bnb_numu_cosmic/sample/numu_corsika_mini_raw.h5'
if not IS_S3DF:
    DATA_PATH = '/sdf/group/neutrino/icarus/bnb_numu_cosmic/sample/numu_corsika_mini_raw.h'
analysis_config = yaml.safe_load(open(ANA_CONFIG, 'r').read().replace('DATA_PATH', DATA_PATH))

cfg = yaml.load(open(MODEL_CONFIG, 'r'), Loader=yaml.Loader)
process_config(cfg, verbose=False)

# CRT-TPC matching config
matcha_cfg_path = '{}/default_matcha.yaml'.format(MATCHA_DIR)


Config processed at: Linux sdfampere001 4.18.0-372.32.1.el8_6.x86_64 #1 SMP Fri Oct 7 12:35:10 EDT 2022 x86_64 x86_64 x86_64 GNU/Linux

$CUDA_VISIBLE_DEVICES="0"



In [6]:
from analysis.manager import AnaToolsManager
manager = AnaToolsManager(analysis_config)

manager.initialize()

Welcome to JupyROOT 6.22/08
Registered /sdf/data/neutrino/icarus/bnb_numu_cosmic/sample/numu_corsika_mini_raw.h5


In [7]:
data, result = manager.forward(iteration=0)

In [8]:
result.keys()

dict_keys(['shower_fragment_node_pred', 'ppn_coords', 'particle_seg', 'particle_node_pred_type', 'particle_start_points', 'track_fragment_end_points', 'ppn_masks', 'track_fragment_start_points', 'fragment_clusts', 'particle_group_pred', 'ppn_classify_endpoints', 'ppn_points', 'segmentation', 'shower_fragment_start_points', 'particle_node_pred_vtx', 'particle_end_points', 'input_rescaled', 'fragment_seg', 'cluster_label_adapted', 'particle_clusts'])

In [9]:
data.keys()
if 'crthits' not in data.keys():
    raise ValueError('crthits not found in data dictionary.')

In [10]:
manager.load_representations(data, result) # This will save 'Particle' and 'Interaction' instances to result dict directly

Building particles instead of loading...
Building interactions instead of loading...


In [11]:
result.keys()

dict_keys(['shower_fragment_node_pred', 'ppn_coords', 'particle_seg', 'particle_node_pred_type', 'particle_start_points', 'track_fragment_end_points', 'ppn_masks', 'track_fragment_start_points', 'fragment_clusts', 'particle_group_pred', 'ppn_classify_endpoints', 'ppn_points', 'segmentation', 'shower_fragment_start_points', 'particle_node_pred_vtx', 'particle_end_points', 'input_rescaled', 'fragment_seg', 'cluster_label_adapted', 'particle_clusts', 'particles', 'interactions', 'truth_particles', 'truth_interactions'])

In [12]:
def has_crthit_match(interactions):
    found_crt_match = False
    interactions_with_match = []
    for interaction_list in interactions:
        for interaction in interaction_list:
            if interaction.crthit_matched:
                found_crt_match = True
                interactions_with_match.append(interaction.id)
                break

    if found_crt_match:
        print('CRT match found in interactions:', interactions_with_match)
    else:
        print('No interactions found with CRT hit match')

In [13]:
has_crthit_match(result['interactions'])

No interactions found with CRT hit match


In [14]:
manager.run_post_processing(data, result)

Running CRT matching...
*******Running with config********
 dca_parameters:
  isdata: false
  method: simple
  threshold: 100
  trigger_timestamp: None
file_save_config:
  save_file_name: matcha_output_test.pkl
  save_file_path: /sdf/data/neutrino/amogan/matcha/
  save_to_file: true
match_making_parameters:
  matching_method: dca
pca_parameters:
  direction_method: pca
  min_points_in_radius: 10
  radius: 10

**********************************
matcha output saved to /sdf/data/neutrino/amogan/matcha//matcha_output_test.pkl
Done CRT matching.


In [15]:
has_crthit_match(result['interactions'])

CRT match found in interactions: [28]
